In [28]:
import pathlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path_material = pathlib.Path('data\\raw_data.csv')

In [3]:
df_raw = pd.read_csv(path_material, low_memory=False)

In [4]:
df_raw.describe()

,year,population,foreign_exch_reserve,GPD_per_cap,Unemployment,Inflation_rate,Life_length_expectancy,GINI,ARMS_export,ARMS_import,Corruption_perception,Temperature_JAN,Temperature_JUL,suicides_no
count,7248.000000,7.248000e+03,7.176000e+03,7248.000000,7092.000000,7248.000000,7248.000000,4656.000000,4.872000e+03,6.384000e+03,4764.000000,7248.000000,7248.000000,7248.000000
mean,2001.774834,1.743408e+06,3.469842e+10,26893.020251,8.820021,3.470996,77.562979,31.333763,6.209064e+08,2.277538e+08,21.985516,1.868540,19.271537,217.996551
std,10.055130,2.235278e+06,3.911449e+10,18015.963059,4.311936,3.911775,2.718098,3.608685,8.498247e+08,3.152652e+08,26.615663,5.127656,3.454063,425.938170
min,1979.000000,5.900000e+03,9.746575e+07,3861.948500,1.482400,-4.478103,70.865854,23.000000,0.000000e+00,0.000000e+00,3.330000,-16.194000,12.571100,0.000000
25%,1995.000000,3.024238e+05,6.400005e+09,13585.207562,5.856900,1.417741,75.475287,28.200000,2.500000e+07,2.400000e+07,6.100000,-1.093000,16.649850,9.000000
50%,2004.000000,7.447485e+05,2.160458e+10,23102.007735,7.865600,2.452733,77.636585,31.350000,2.080000e+08,1.045000e+08,8.000000,2.732625,18.887950,63.000000
75%,2010.000000,2.538383e+06,4.807903e+10,35575.159547,10.554500,4.053958,79.793293,34.300000,9.510000e+08,2.865000e+08,42.000000,5.044645,21.743575,214.000000
max,2016.000000,1.314881e+07,2.490000e+11,118823.648400,27.466200,24.506067,83.229268,39.000000,4.037000e+09,2.167000e+09,92.000000,22.723500,29.786000,3427.000000


Define function to change all titles to lower cases along with substituting and dropping several uncommon characters.

In [5]:
def refactor_titles(df: pd.DataFrame, sub_chars=[' ','-',':'], drop_chars=['(',')','[',']']) -> list:
    '''
    Function generalizes all column titles, i.e. lowers all cases, makes characters 
    substitution from one input list and drops characters from another input list.
    (1) df --> DataFrame which columns function should transform;
    (2) sub_chars --> characters to substitute with '_';
    (3) drop_chars --> characters to drop;
    Returns list with new column titles.
    '''
    list_columns = [item.lower() for item in (df.columns.tolist())]   
    for sub_char in sub_chars:
        counter_sub = 0
        for i in range(len(list_columns)):
            if sub_char in list_columns[i]:
                counter_sub += 1
                list_columns[i] = list_columns[i].replace(sub_char, '_')
            else:
                continue
        print(f'Substituotion of "{sub_char}" occured {counter_sub} times.')
    for drop_char in drop_chars:
        counter_drop = 0
        for i in range(len(list_columns)):
            if drop_char in list_columns[i]:
                list_columns[i] = list_columns[i].replace(drop_char, '') 
            else:
                continue
            print(f'Drop of "{drop_char}" occured {counter_drop} times.')              
    return list_columns

Initialize **refactor_titles()** function on the DataFrame's columns

In [6]:
df_raw.columns = refactor_titles(df_raw)

Substituotion of " " occured 0 times.
Substituotion of "-" occured 0 times.
Substituotion of ":" occured 0 times.


After 2012 Corruption perception evaluation changes from range 10 to range 100.
In order to generalize column, all values after 2012 will be divided by 10.

In [7]:
df_raw['corruption_perception'] = df_raw['corruption_perception'].apply(lambda x: x/10 if x > 10 else x)

Check the DataFrame for null-values.

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7248 entries, 0 to 7247
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 7248 non-null   object 
 1   year                    7248 non-null   int64  
 2   sex                     7248 non-null   object 
 3   age                     7248 non-null   object 
 4   population              7248 non-null   int64  
 5   foreign_exch_reserve    7176 non-null   float64
 6   gpd_per_cap             7248 non-null   float64
 7   unemployment            7092 non-null   float64
 8   inflation_rate          7248 non-null   float64
 9   life_length_expectancy  7248 non-null   float64
 10  gini                    4656 non-null   float64
 11  arms_export             4872 non-null   float64
 12  arms_import             6384 non-null   float64
 13  corruption_perception   4764 non-null   float64
 14  temperature_jan         7248 non-null   

All empty values in both ARMS import and ARMS export columns will be filled with '0', as these countries didn't participate neither in ARMS import nor in ARMS export.

In [9]:
df_raw['arms_import'].fillna(0, inplace=True)
df_raw['arms_export'].fillna(0, inplace=True)

Derive column names containing at least one NaN value, put them into list and assign to variable.

In [10]:
columns_nan_list = df_raw.columns[df_raw.isna().any()].tolist()

Define function to attach country names where NaN appeared and return dictionary with countries.

In [11]:
def nan_columns_by_country(columns_nan_list: list,) -> dict:
    set_of_nan = {}
    for column in columns_nan_list:
        set_of_nan[column] = list(set(df_raw[df_raw[column].isnull()]['country']))
    for key, value in sorted(set_of_nan.items()):
        print(f'* {key}: {len([item for item in value if item])} countries with at least one NaN.')
    return set_of_nan

Initialize function **nan_columns_by_country()**.

In [21]:
columns_nan_list = nan_columns_by_country(columns_nan_list)

* corruption_perception: 22 countries with at least one NaN.
* foreign_exch_reserve: 0 countries with at least one NaN.
* gini: 16 countries with at least one NaN.
* unemployment: 0 countries with at least one NaN.


Function returns 4 columns: foreign_exch_reserve | unemployment | gini | corruption_perception.
Let's take 'foreign_exch_reserve' & 'unemployment' as only several countries have missing values.

In [23]:
fer_list = columns_nan_list['foreign_exch_reserve']#; print(fer_list)
unemployment_list = columns_nan_list['unemployment']#; print(unemployment_list)

Define function to fill NaN values using .fillna() with 'backfill' method.

In [16]:
def backfilling_nan_by_country(df: pd.DataFrame,nan_list: list, column_df: str):
    for country in nan_list:
        mask = df['country'] == country
        df.loc[mask,column_df] = df.loc[mask,column_df].fillna(method='backfill')

Fill 'unemployment' column.

In [17]:
backfilling_nan_by_country(df_raw,unemployment_list,'unemployment')

Fill 'foreign_exch_reserve' column.

In [18]:
backfilling_nan_by_country(df_raw,fer_list,'foreign_exch_reserve')

'gini' and 'corruption_perception' columns still remain with NaN, but next let's deal with very low cordinality columns categorization to implement LabelEncoder().

In [56]:
very_low_cardinality_cols = [col_name for col_name in df_raw.columns if df_raw[col_name].nunique() < 5 and 
                                                                   df_raw[col_name].dtype == 'object']
very_low_cardinality_cols

['sex']

In [57]:
from sklearn.preprocessing import LabelEncoder

label_df_raw = df_raw.copy()
label_encoder = LabelEncoder()
for col in very_low_cardinality_cols:
    label_df_raw[col] = label_encoder.fit_transform(df_raw[col])

In [62]:
label_df_raw.head(12)

,country,year,sex,age,population,foreign_exch_reserve,gpd_per_cap,unemployment,inflation_rate,life_length_expectancy,gini,arms_export,arms_import,corruption_perception,temperature_jan,temperature_jul,suicides_no
0,Austria,1995,0,15-24 years,495368,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,19
1,Austria,1995,0,25-34 years,678262,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,59
2,Austria,1995,0,35-54 years,1065630,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,136
3,Austria,1995,0,5-14 years,460945,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,0
4,Austria,1995,0,55-74 years,843210,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,141
5,Austria,1995,0,75+ years,341098,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,100
6,Austria,1995,1,15-24 years,509262,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,134
7,Austria,1995,1,25-34 years,707468,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,237
8,Austria,1995,1,35-54 years,1074328,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,437
9,Austria,1995,1,5-14 years,483644,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,0.0,24000000.0,NaN,-3.3684,18.3665,8


In [59]:
low_cardinality_cols = [col_name for col_name in df_raw.columns if 5 < df_raw[col_name].nunique() < 10 and 
                                                                   df_raw[col_name].dtype == 'object']
low_cardinality_cols

['age']

In [65]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) # Apply one-hot encoder to each column with categorical data
OH_cols_df = pd.DataFrame(OH_encoder.fit_transform(df_raw[low_cardinality_cols]))

OH_cols_df.index = df_raw.index # One-hot encoding removed index; put it back
OH_cols_df.rename(columns = {0: '15-24 years',
                             1: '25-34 years',
                             2: '35-54 years',
                             3: '5-14 years',
                             4: '55-74 years',
                             5: '75+ years'}, inplace=True)

num_df = df_raw.drop(low_cardinality_cols, axis=1) # Remove categorical columns (will replace with one-hot encoding)
OH_df = pd.concat([num_df,OH_cols_df], axis=1) # Add one-hot encoded columns to numerical features

In [66]:
OH_df

,country,year,sex,population,foreign_exch_reserve,gpd_per_cap,unemployment,inflation_rate,life_length_expectancy,gini,...,corruption_perception,temperature_jan,temperature_jul,suicides_no,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years
0,Austria,1995,female,495368,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,...,NaN,-3.36840,18.3665,19,1.0,0.0,0.0,0.0,0.0,0.0
1,Austria,1995,female,678262,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,...,NaN,-3.36840,18.3665,59,0.0,1.0,0.0,0.0,0.0,0.0
2,Austria,1995,female,1065630,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,...,NaN,-3.36840,18.3665,136,0.0,0.0,1.0,0.0,0.0,0.0
3,Austria,1995,female,460945,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,...,NaN,-3.36840,18.3665,0,0.0,0.0,0.0,1.0,0.0,0.0
4,Austria,1995,female,843210,2.336874e+10,30325.84958,4.3473,2.243367,76.668293,31.1,...,NaN,-3.36840,18.3665,141,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7243,United Kingdom,2015,male,4408293,1.480000e+11,44974.83188,5.3003,0.368047,80.956098,33.2,...,8.1,3.87941,14.5338,596,0.0,1.0,0.0,0.0,0.0,0.0
7244,United Kingdom,2015,male,8692440,1.480000e+11,44974.83188,5.3003,0.368047,80.956098,33.2,...,8.1,3.87941,14.5338,1579,0.0,0.0,1.0,0.0,0.0,0.0
7245,United Kingdom,2015,male,3840476,1.480000e+11,44974.83188,5.3003,0.368047,80.956098,33.2,...,8.1,3.87941,14.5338,4,0.0,0.0,0.0,1.0,0.0,0.0
7246,United Kingdom,2015,male,6716557,1.480000e+11,44974.83188,5.3003,0.368047,80.956098,33.2,...,8.1,3.87941,14.5338,906,0.0,0.0,0.0,0.0,1.0,0.0


In [37]:
# print(df_raw.groupby('country').size().sort_values(ascending=False))

In [68]:
msk = np.random.rand(len(df_raw)) < 0.8

array([ True,  True, False, ..., False, False,  True])

In [74]:
print(len(df_raw[~msk]))
print(len(df_raw[msk]))
print(len(df_raw[msk])/len(df_raw))

1434
5814
0.8021523178807947


In [20]:
### To-Do list:
# * Catboost for countries;
# *
# *
# *